In [1]:
!pip install optuna

   ---------------------------------------- 0.0/362.8 kB ? eta -:--:--
   ---------------- ----------------------- 153.6/362.8 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 362.8/362.8 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/233.2 kB ? eta -:--:--
   ---------------------------------------- 233.2/233.2 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.6 kB ? eta -:--:--
   ---------------------------------------- 78.6/78.6 kB 4.3 MB/s eta 0:00:00


In [2]:
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
iris = load_iris()
X = iris.data
y = iris.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)

In [5]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
    rf = RandomForestClassifier(n_estimators=n_estimators, 
                                max_depth=max_depth, 
                                min_samples_split=min_samples_split, 
                                random_state=42)
    
    score = cross_val_score(rf, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [6]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-10-06 17:47:50,841] A new study created in memory with name: no-name-686954b8-e366-4bd6-b3e0-796fc970f00e
[I 2024-10-06 17:47:51,298] Trial 0 finished with value: 0.9333333333333332 and parameters: {'n_estimators': 74, 'max_depth': 9, 'min_samples_split': 5}. Best is trial 0 with value: 0.9333333333333332.
[I 2024-10-06 17:47:51,763] Trial 1 finished with value: 0.9333333333333332 and parameters: {'n_estimators': 76, 'max_depth': 13, 'min_samples_split': 4}. Best is trial 0 with value: 0.9333333333333332.
[I 2024-10-06 17:47:52,460] Trial 2 finished with value: 0.9238095238095237 and parameters: {'n_estimators': 113, 'max_depth': 17, 'min_samples_split': 3}. Best is trial 0 with value: 0.9333333333333332.
[I 2024-10-06 17:47:53,275] Trial 3 finished with value: 0.9333333333333332 and parameters: {'n_estimators': 135, 'max_depth': 6, 'min_samples_split': 10}. Best is trial 0 with value: 0.9333333333333332.
[I 2024-10-06 17:47:53,394] Trial 4 finished with value: 0.91428571428571

In [7]:
best_params = study.best_params
rf_optimized = RandomForestClassifier(**best_params, random_state=42)
rf_optimized.fit(X_train, y_train)


RandomForestClassifier(max_depth=9, min_samples_split=5, n_estimators=74,
                       random_state=42)

In [8]:
y_pred_optimized = rf_optimized.predict(X_test)
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)

print(f"Melhores hiperparâmetros: {best_params}")
print(f"Precisão do modelo otimizado: {accuracy_optimized:.2f}")

Melhores hiperparâmetros: {'n_estimators': 74, 'max_depth': 9, 'min_samples_split': 5}
Precisão do modelo otimizado: 1.00
